In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline


class Config:
    def __init__(self):
        pass


def set_config(config_dict):
    config = Config()
    config.__dict__ = config_dict
    return config


In [4]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

In [6]:
sigmoid(1)

0.7310585786300049

In [7]:
tanh(1)

0.7615941559557649

In [2]:
config = set_config({
    'row_start': 7228,
    'row_end': 7288,
    "timestep": 2,
    "max_batch_size": 1,
    "layer_size": 1,
    "unit_size": 1,
    "dropout": 0,
    "learning_rate": 0.1,
    "max_epoch": 50,
    "time_col": 'tanggal',
    "feature": ['rr']
})

# usecols = [*[config.time_col], *config.feature]


def train_test_split(dataset, timestep=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-timestep):
        dataX.append(dataset[i:(i+timestep), 0])
        dataY.append(dataset[i+timestep, 0])
    return np.array(dataX), np.array(dataY)


In [3]:
dataset = pd.read_csv('../Data/1985-2021.csv').replace(to_replace=[8888, 9999, 2555], value=np.nan)[config.row_start:config.row_end]
dataset.interpolate(inplace=True)

datelist = [datetime.strptime(date, '%Y-%m-%d').date() for date in list(dataset[config.time_col])]

In [4]:
featureset = dataset[config.feature]
vector_featureset = featureset.values

scaller = MinMaxScaler()
vector_featureset_scaled = scaller.fit_transform(vector_featureset)

train_size = int(vector_featureset_scaled.size * 0.9)
trainset, testset = vector_featureset_scaled[0:train_size], vector_featureset_scaled[train_size:vector_featureset_scaled.size]

X_train, y_train = train_test_split(trainset, timestep=config.timestep)
X_train = np.reshape(X_train, (X_train.shape[0], len(config.feature), X_train.shape[1]))
# y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1]))

X_test, y_test = train_test_split(testset, timestep=config.timestep)
X_test = np.reshape(X_test, (X_test.shape[0], len(config.feature), X_test.shape[1]))
# y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1]))

In [5]:
y_pred_loss = list()
y_true_loss = list()


def mean_squared_error(y_true, y_pred):
    y_pred_loss.append(y_pred)
    y_true_loss.append(y_true)
    return tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true))


def root_mean_squared_error(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(mean_squared_error(y_pred - y_true)))

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.LSTM(
    config.unit_size,
    batch_input_shape=(config.max_batch_size, len(config.feature), config.timestep),
    go_backwards=True,
    dropout=config.dropout,
    weights=[
        np.float32([
            [0.5774, 0.5774, 0.5774, 0.5774],
            [0.5774, 0.5774, 0.5774, 0.5774]
        ]),
        np.float32([
            [0.5774, 0.5774, 0.5774, 0.5774]
        ]),
        np.float32(
            [0, 0, 0, 0]
        )
    ],
))

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=config.learning_rate),
    loss=mean_squared_error,
    run_eagerly=True
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (1, 1)                    16        
                                                                 
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [6]:
batch_loss = list()
        
history = model.fit(
    X_train, 
    y_train, 
    shuffle=False, 
    epochs=config.max_epoch, 
    verbose=1, 
    batch_size=config.max_batch_size, 
    callbacks=[
        tf.keras.callbacks.LambdaCallback(
            on_epoch_begin=None,
            on_epoch_end=None,
            on_batch_begin=None,
            on_batch_end=lambda batch, logs=None: batch_loss.append(logs['loss']),
            on_train_begin=None,
            on_train_end=None,
        )
    ]
)

Epoch 1/50
52/52 [==============================] - 3s 52ms/step - loss: 0.0797
Epoch 2/50
52/52 [==============================] - 2s 35ms/step - loss: 0.0722
Epoch 3/50
52/52 [==============================] - 2s 37ms/step - loss: 0.0698
Epoch 4/50
52/52 [==============================] - 2s 37ms/step - loss: 0.0687
Epoch 5/50
52/52 [==============================] - 3s 59ms/step - loss: 0.0680
Epoch 6/50
52/52 [==============================] - 2s 44ms/step - loss: 0.0676
Epoch 7/50
52/52 [==============================] - 2s 39ms/step - loss: 0.0672
Epoch 8/50
52/52 [==============================] - 2s 37ms/step - loss: 0.0669
Epoch 9/50
52/52 [==============================] - 2s 38ms/step - loss: 0.0666
Epoch 10/50
52/52 [==============================] - 2s 36ms/step - loss: 0.0664
Epoch 11/50
52/52 [==============================] - 2s 37ms/step - loss: 0.0662
Epoch 12/50
52/52 [==============================] - 3s 56ms/step - loss: 0.0660
Epoch 13/50
52/52 [==================

In [7]:
batch_loss

[{'loss': 0.9924630522727966},
 {'loss': 0.5416995882987976},
 {'loss': 0.3870604932308197},
 {'loss': 0.2904958426952362},
 {'loss': 0.23416030406951904},
 {'loss': 0.19666725397109985},
 {'loss': 0.16890378296375275},
 {'loss': 0.14811253547668457},
 {'loss': 0.13364611566066742},
 {'loss': 0.12047319114208221},
 {'loss': 0.1137012168765068},
 {'loss': 0.10453721880912781},
 {'loss': 0.10512326657772064},
 {'loss': 0.09761984646320343},
 {'loss': 0.09150534123182297},
 {'loss': 0.08656332641839981},
 {'loss': 0.08227597177028656},
 {'loss': 0.07913236320018768},
 {'loss': 0.07658860832452774},
 {'loss': 0.07337412238121033},
 {'loss': 0.07078462839126587},
 {'loss': 0.06784328818321228},
 {'loss': 0.06561852991580963},
 {'loss': 0.06305984407663345},
 {'loss': 0.07602135092020035},
 {'loss': 0.07381964474916458},
 {'loss': 0.07150454819202423},
 {'loss': 0.07906987518072128},
 {'loss': 0.07637372612953186},
 {'loss': 0.07708169519901276},
 {'loss': 0.0757325068116188},
 {'loss': 0.07

In [8]:
model.predict(X_test, verbose=1, batch_size=config.max_batch_size)

4/4 [==============================] - 0s 10ms/step


array([[0.2002306 ],
       [0.18857674],
       [0.18828028],
       [0.18841618]], dtype=float32)